In [163]:
class Code:
    jump_codes = {'': '000', 
                  'JGT': '001', 
                  'JEQ': '010', 
                  'JGE': '011', 
                  'JLT': '100', 
                  'JNE': '101', 
                  'JLE': '110', 
                  'JMP': '111'}
    dest_codes = {'':'000', 
                  'M':'001', 
                  'D':'010', 
                  'MD':'011', 
                  'A':'100', 
                  'AM':'101', 
                  'AD':'110', 
                  'AMD':'111'}
    comp_codes = {'0':'0101010',  '1':'0111111',  '-1':'0111010', 'D':'0001100', 
                'A':'0110000',  '!D':'0001101', '!A':'0110001', '-D':'0001111', 
                '-A':'0110011', 'D+1':'0011111','A+1':'0110111','D-1':'0001110', 
                'A-1':'0110010','D+A':'0000010','D-A':'0010011','A-D':'0000111', 
                'D&A':'0000000','D|A':'0010101',
                '':'xxxxxxx',   '':'xxxxxxx',   '':'xxxxxxx',   '':'xxxxxxx', 
                'M':'1110000',  '':'xxxxxxx',   '!M':'1110001', '':'xxxxxxx', 
                '-M':'1110011', '':'xxxxxxx',   'M+1':'1110111','':'xxxxxxx', 
                'M-1':'1110010','D+M':'1000010','D-M':'1010011','M-D':'1000111', 
                'D&M':'1000000', 'D|M':'1010101' }
    
    @classmethod
    def conv_comp(cls, field):
        return cls.comp_codes[field]
    
    @classmethod
    def conv_dest(cls, field):
        return cls.dest_codes[field]
    
    @classmethod
    def conv_jump(cls, field):
        return cls.jump_codes[field]

    
class SymbolTable:
    _symbol_table = {'SP':0, 'LCL':1, 'ARG':2, 'THIS':3, 'THAT':4,
               'R0':0, 'R1':1, 'R2':2, 'R3':3, 'R4':4, 'R5':5, 'R6':6, 'R7':7,
               'R8':8, 'R9':9, 'R10':10, 'R11':11, 'R12':12, 'R13':13, 'R14':14, 'R15':15,
               'SCREEN':16384, 'KBD':24576}
    n=16
    
    def populate(self, insts):
        i = 0
        for inst in insts:
            if inst[0] == '(':
                symbol = inst[1:-1]
                self._symbol_table[symbol] = i
            else:
                i+=1
        
    def second_pass_add(self, symbol):
        self._symbol_table[symbol] = self.n
        self.n += 1
        return self._symbol_table[symbol]
        
    def get(self, symbol):
        if symbol in self._symbol_table:
            value = self._symbol_table[symbol]
        else:
            value = self.second_pass_add(symbol)
        return value
        
                

class Parser:
    symbol_table = None
    def prepare_inst(self, inst):
        inst = inst.replace(" ", "")
        if '//' in inst:
            inst = inst.split('//')[0]
        return inst
    
    def conv_inst(self, inst):
        """
            MD=D+1 -> 1110011111011000
            @43 -> 0000000000101011
        """
        if inst[0] == '@':
            bin_inst = self.conv_a_inst(inst)
        else:
            bin_inst = self.conv_c_inst(inst)
        assert len(bin_inst) == 16
        return bin_inst
            
    def conv_a_inst(self, inst):
        # "@43 -> 0000000000101011"
        # "@kek -> 0000000000101011"
        symbol = inst[1:]
        if symbol.isdigit():
            value = symbol
        else:
            value = self.symbol_table.get(symbol)
        return '0' + '{:015b}'.format(int(value) % 32768)
    
    def parse_c_inst(self, inst):
        # D = D + 1 ; JMP // it is ok func
        # D = D + 1 // it is ok func
        # D + 1 ; JMP // it is ok func
        if ';' in inst:
            inst, jump = inst.split(';')
        else:
            jump = ''
        if '=' in inst:
            dest, comp = inst.split('=')
        else:
            dest = ''
            comp = inst
        return dest, comp, jump
    
    
    def conv_c_inst(self, inst):
        # MD=D+1 -> 1110011111011000
        # D=D+1;JMP -> 1110011111010111
        # D = D + 1 ; JMP // it is ok func
        # D = D + 1 // it is ok func
        # D + 1 ; JMP // it is ok func
        dest, comp, jump = self.parse_c_inst(inst)
        return '111' + Code.conv_comp(comp) + Code.conv_dest(dest) + Code.conv_jump(jump)

In [165]:
def assemble_file(file_path):
    with open(file_path) as file:
        insts = file.read().splitlines()
    print(insts)
    parser = Parser()
    insts = [parser.prepare_inst(x) for x in insts] # delete whitespaces and comments
    insts = [x for x in insts if x != ''] # filter empty
    print(insts)
    
    st = SymbolTable()
    st.populate(insts) # first iter: add (LABELS)
    parser.symbol_table = st
    
    insts = [x for x in insts if x[0]!='('] # filter (LABELS)
    
    insts = [parser.conv_inst(x) for x in insts]
    print(insts)
    with open(file_path.split('.')[0] + '.hack', "w") as outfile:
        outfile.write("\n".join(insts))

In [167]:
for file in ['add/Add.asm', 'max/MaxL.asm', 'max/Max.asm']:
    assemble_file(file)
    
for file in ['pong/PongL.asm', 'rect/RectL.asm', 'pong/Pong.asm', 'rect/Rect.asm']:
    assemble_file(file)   